In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd 


In [2]:
df = pd.read_csv('./location_and_label_rmNan.csv',encoding='CP949')
df

,No.,Location,ID,위험정도,일반영상,열화상영상,레이블영상,주소
0,1,wadong-dong,1,1,RGB0,THR0,LAB0,경기 파주시 와동동 1517
1,2,wadong-dong,1,1,RGB1,THR1,LAB1,경기 파주시 와동동 1517
2,3,wadong-dong,1,1,RGB2,THR2,LAB2,경기 파주시 와동동 1517
3,4,wadong-dong,1,1,RGB3,THR3,LAB3,경기 파주시 와동동 1517
4,5,wadong-dong,1,1,RGB4,THR4,LAB4,경기 파주시 와동동 1517
...,...,...,...,...,...,...,...,...
9995,9996,"seonghwa, gaesin, jungnim-dong",17,2,RGB9995,THR9995,LAB9995,충북 청주시 서원구 산남동 683
9996,9997,"seonghwa, gaesin, jungnim-dong",17,2,RGB9996,THR9996,LAB9996,충북 청주시 서원구 산남동 683
9997,9998,"seonghwa, gaesin, jungnim-dong",17,1,RGB9997,THR9997,LAB9997,충북 청주시 서원구 산남동 683
9998,9999,"seonghwa, gaesin, jungnim-dong",17,1,RGB9998,THR9998,LAB9998,충북 청주시 서원구 산남동 683


In [3]:
val = df.loc[:,["위험정도","레이블영상"]].values
danger,labPaths = val[:,0],val[:,1]
danger[:10],labPaths[:10]
dir = "targetData3"
rgbs = [os.path.join(dir,"R"+lab[3:]+'.png') for lab in labPaths]
thrs = [os.path.join(dir,"T"+lab[3:]+'.png') for lab in labPaths]
labs = [os.path.join(dir,lab+'.png') for lab in labPaths]
rgbs[:10],thrs[:10],labs[:10]
i = 0
while i < len(rgbs):
    if not os.path.exists(rgbs[i]) or not os.path.exists(thrs[i]) or not os.path.exists(labs[i]):
        print()
        rgbs.pop(i)
        thrs.pop(i)
        labs.pop(i)
        danger = np.delete(danger,[i])
    i+=1

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset

from torchvision import transforms
from PIL import Image
import cv2 

imgtrans = transforms.Compose([
    transforms.Resize(size=(128,256)),
    transforms.ToTensor()
])

class CompleteRgbThrDataset(torch.utils.data.Dataset):
    def __init__(self,rgbList,thrList,labList,label,imgtrans = imgtrans):
        self.rgbList = rgbList
        self.thrList = thrList
        self.labList = labList
        self.imgtrans = imgtrans
        self.label = label 
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self,idx):
        rgb = Image.open(self.rgbList[idx]).convert('RGB')
        thr = Image.open(self.thrList[idx]).convert('RGB')
        # lab = Image.fromarray((np.array(Image.open(self.labList[idx]).convert('L'))>=127.0).astype(np.uint8)*255)
        lab = Image.open(self.labList[idx]).convert('L')
        label = torch.LongTensor([self.label[idx]])
        if self.imgtrans:
            rgb = self.imgtrans(rgb)
            thr = self.imgtrans(thr)
            lab = self.imgtrans(lab)
        lab = (lab>=0.5).type(torch.float32)
        
        return rgb,thr,lab,label 




In [6]:
from models.MBConvNet import MBConvIntertwinedUNet
from models.FuseNet import FuseNet
from models.MFNet import MFNet
from models.UNet import UNet
from models.SegNet import SegNet
from models.RTFNet import RTFNet 
fuseNet = FuseNet(1)
mfNet = MFNet(1)
uNet = UNet(6,1)
segNet = SegNet(1)
rtfNet = RTFNet(1)
mbConvNet = MBConvIntertwinedUNet(1)



RuntimeError: CUDA error: out of memory